In [1]:
import pandas as pd
links=pd.read_csv("ml-32m/links.csv")

In [2]:
print(links.head(),links.shape)
links.drop(labels='imdbId',axis=1,inplace=True)#,'movieId'
print('number of duplicated recored in each column\n',links[links.duplicated('tmdbId')].count(),links[links.duplicated('movieId')].count())
links.drop_duplicates(subset=['movieId'],inplace=True)
links.drop_duplicates(subset=['tmdbId'],inplace=True)
print(links[links.duplicated()])
print('work on duplincation ends \n------------------------------------------------')
print('count na data \n',links.isna().sum())
links.dropna(inplace=True)
print('count na data after drop\n',links.isna().sum(),'\n------------------------------------------------')


   movieId  imdbId   tmdbId
0        1  114709    862.0
1        2  113497   8844.0
2        3  113228  15602.0
3        4  114885  31357.0
4        5  113041  11862.0 (87585, 3)
number of duplicated recored in each column
 movieId    159
tmdbId      36
dtype: int64 movieId    0
tmdbId     0
dtype: int64
Empty DataFrame
Columns: [movieId, tmdbId]
Index: []
work on duplincation ends 
------------------------------------------------
count na data 
 movieId    0
tmdbId     1
dtype: int64
count na data after drop
 movieId    0
tmdbId     0
dtype: int64 
------------------------------------------------


In [3]:
import os 
lpath=r'cpdata1v2'
os.makedirs(lpath,exist_ok=True)
links.to_csv(lpath+'\\links.csv',index=False)


In [4]:
links=pd.read_csv(lpath+'\\links.csv')
rating=pd.read_csv(lpath+'\\ratings.csv')
movies=pd.read_csv(lpath+'\\movies_data.csv')
print(links.shape,rating.shape,movies.shape)

(87425, 2) (32000204, 4) (18460, 20)


In [5]:
print(rating[rating.duplicated()])
print(rating.isna().sum())

Empty DataFrame
Columns: [userId, movieId, rating, timestamp]
Index: []


userId       0
movieId      0
rating       0
timestamp    0
dtype: int64

In [6]:
print(movies.shape)
print('check duplication\n',movies[movies.duplicated()],'\n-----------------------------------')
print('check duplication\n',movies[movies['Movie ID'].duplicated()],'\n-----------------------------------')
print('check missing value in each column\n',movies.isna().sum(),'\n-----------------------------------')

(18460, 20)
check duplication
 Empty DataFrame
Columns: [Movie ID, Title, Overview, Genres, Release Date, Runtime, Average Rating, Vote Count, Popularity, Budget, Adult, Poster Path, Backdrop Path, Cast, Director, Keywords, Production Companies, Production Countries, Similar Movies, Movie Homepage]
Index: [] 
-----------------------------------
check duplication
 Empty DataFrame
Columns: [Movie ID, Title, Overview, Genres, Release Date, Runtime, Average Rating, Vote Count, Popularity, Budget, Adult, Poster Path, Backdrop Path, Cast, Director, Keywords, Production Companies, Production Countries, Similar Movies, Movie Homepage]
Index: [] 
-----------------------------------
check missing value in each column
 Movie ID                    0
Title                       0
Overview                  521
Genres                    286
Release Date               27
Runtime                     0
Average Rating              0
Vote Count                  0
Popularity                  0
Budget      

In [7]:
movies.dropna(axis=0,subset=['Cast','Director','Overview','Genres','Poster Path','Backdrop Path','Release Date'],inplace=True)

In [8]:
print(links.shape)
links=links[links['tmdbId'].isin(movies['Movie ID'])]
links=links[links['movieId'].isin(rating['movieId'])]
print(links.shape)
movies=movies[movies['Movie ID'].isin(links['tmdbId'])]
rating=rating[rating['movieId'].isin(links['movieId'])]
print(len(rating['movieId'].unique()),len(rating['userId'].unique()))
rating.shape,movies.shape


(87425, 2)
(13663, 2)
13663 200948


((29606259, 4), (13663, 20))

In [9]:
print('check duplication\n',movies[movies.duplicated()],'\n-----------------------------------')
print('check missing value in each column\n',movies.isna().sum(),'\n-----------------------------------')

check duplication
 Empty DataFrame
Columns: [Movie ID, Title, Overview, Genres, Release Date, Runtime, Average Rating, Vote Count, Popularity, Budget, Adult, Poster Path, Backdrop Path, Cast, Director, Keywords, Production Companies, Production Countries, Similar Movies, Movie Homepage]
Index: [] 
-----------------------------------
check missing value in each column
 Movie ID                   0
Title                      0
Overview                   0
Genres                     0
Release Date               0
Runtime                    0
Average Rating             0
Vote Count                 0
Popularity                 0
Budget                     0
Adult                      0
Poster Path                0
Backdrop Path              0
Cast                       0
Director                   0
Keywords                 797
Production Companies     325
Production Countries      71
Similar Movies           653
Movie Homepage          9386
dtype: int64 
-----------------------------------

In [10]:
lpath=r'cpdata2v2'
os.makedirs(lpath,exist_ok=True)
links.to_csv(lpath+'\\links.csv',index=False)
rating.to_csv(lpath+'\\ratings.csv',index=False)
movies.to_csv(lpath+'\\movies.csv',index=False)

In [18]:
lpath=r'cpdata2v2'
rating=pd.read_csv(lpath+'\\ratings.csv')
movies=pd.read_csv(lpath+'\\movies.csv')
links=pd.read_csv(lpath+'\\links.csv')
rating.shape,movies.shape,links.shape

((29606259, 4), (13663, 20), (13663, 2))

In [19]:
print(rating.shape)
temp=rating.groupby('movieId').filter(lambda x:len(x)>=50)
print(temp.shape)
temp=temp.groupby('userId').filter(lambda x:len(x)>=35)
print(temp.shape)

(29606259, 4)
(29538106, 4)
(28289032, 4)


In [20]:
rating=temp
rating.shape

(28289032, 4)

In [21]:
links=links[links['movieId'].isin(rating['movieId'])]
movies=movies[movies['Movie ID'].isin(links['tmdbId'])]
print('movie: ',len(movies['Movie ID'].unique()),'rating movie :',len(rating['movieId'].unique()))

movie:  9447 rating movie : 9447


In [22]:
r2t=dict({})
t2m=dict({})
x=1
for index,row in movies.iterrows():
    if t2m.get(row["Movie ID"])==None:
        t2m[row["Movie ID"]]=x
        x+=1
    else:
        print("there is redundancy")
        break
for index,row in links.iterrows():
    if r2t.get(row["movieId"])==None:
        r2t[row["movieId"]]=row['tmdbId']
    else:
        print("there is redundancy")
rating['movieId'] = rating['movieId'].map(lambda x: t2m[r2t[x]])
movies['Movie ID']=movies['Movie ID'].map(lambda x:t2m[x])
rating=rating.drop(labels=['timestamp'],axis=1)
rating['userId'] =pd.factorize(rating['userId'])[0]+1
len(rating['userId'].unique()),len(rating['movieId'].unique())

(151967, 9447)

In [23]:
lpath=r'cpdata3v2'
os.makedirs(lpath,exist_ok=True)
links.to_csv(lpath+'\\links.csv',index=False)
rating.to_csv(lpath+'\\ratings.csv',index=False)
movies.to_csv(lpath+'\\movies.csv',index=False)

In [24]:
lpath=r'cpdata3v2'
rating=pd.read_csv(lpath+'\\ratings.csv')
movies=pd.read_csv(lpath+'\\movies.csv')
links=pd.read_csv(lpath+'\\links.csv')

In [30]:
temp_above_600=rating.groupby('userId').filter(lambda x:len(x)>=600)
print(temp_above_600.shape)
movie_counts = rating['movieId'].value_counts().to_dict()
user_counts  = rating['userId' ].value_counts().to_dict()
ulp = []
for w, (idx, row) in enumerate(temp_above_600.iterrows(), 1):
    user_id  = row['userId']
    movie_id = row['movieId']
    if movie_counts.get(movie_id, 0) > 50 and user_counts.get(user_id, 0) > 600:
        movie_counts[movie_id] -= 1
        user_counts[user_id]  -= 1
        ulp.append((user_id, movie_id))
    if w % 500_000 == 0:
        print(w)
to_remove=pd.MultiIndex.from_tuples(ulp,names=['userId', 'movieId'])
keep_mask = ~rating.set_index(['userId', 'movieId']).index.isin(to_remove)
print(len(ulp))
len(rating[keep_mask].movieId.unique())

(8066061, 3)
500000
1000000
1500000
2000000
2500000
3000000
3500000
4000000
4500000
5000000
5500000
6000000
6500000
7000000
7500000
8000000
3239661


9447

In [33]:
rating=rating[keep_mask]
rating.shape

(25049371, 3)

In [34]:
movies.columns

Index(['Movie ID', 'Title', 'Overview', 'Genres', 'Release Date', 'Runtime',
       'Average Rating', 'Vote Count', 'Popularity', 'Budget', 'Adult',
       'Poster Path', 'Backdrop Path', 'Cast', 'Director', 'Keywords',
       'Production Companies', 'Production Countries', 'Similar Movies',
       'Movie Homepage'],
      dtype='object')

In [36]:
movies.drop(labels=['Average Rating', 'Vote Count','Poster Path', 'Backdrop Path', 'Keywords', 'Similar Movies'],axis=1,inplace=True)

In [37]:
lpath=r'cpdata4v2'
os.makedirs(lpath,exist_ok=True)
links.to_csv(lpath+'\\links.csv',index=False)
rating.to_csv(lpath+'\\ratings.csv',index=False)
movies.to_csv(lpath+'\\movies.csv',index=False)

In [38]:
lpath=r'cpdata4v2'
import pandas as pd 
movies=pd.read_csv(lpath+'\\movies.csv')

In [39]:
movies['Genres']

0            Comedy, Drama, Romance, Crime
1                   Comedy, Drama, Romance
2                            Comedy, Crime
3                  Action, Crime, Thriller
4       Adventure, Action, Science Fiction
                       ...                
9442       Crime, Drama, Thriller, History
9443                        Drama, History
9444        Fantasy, Drama, Comedy, Family
9445               Crime, Action, Thriller
9446               Crime, Action, Thriller
Name: Genres, Length: 9447, dtype: object

In [41]:
genres=pd.DataFrame(columns=['movie_id','genre'])
c=0
for index,row in movies.iterrows():
    for g in row.get("Genres").split(','):
        genres.loc[c]=[row.get("Movie ID"),g.replace(" ","")]
        c+=1

In [42]:
genres

,movie_id,genre
0,1,Comedy
1,1,Drama
2,1,Romance
3,1,Crime
4,2,Comedy
...,...,...
24882,9446,Action
24883,9446,Thriller
24884,9447,Crime
24885,9447,Action


In [43]:
movies.drop(labels='Genres',axis=1,inplace=True)
movies.columns

Index(['Movie ID', 'Title', 'Overview', 'Release Date', 'Runtime',
       'Popularity', 'Budget', 'Adult', 'Cast', 'Director',
       'Production Companies', 'Production Countries', 'Movie Homepage'],
      dtype='object')

In [1]:
lpath=r'cpdata4v2'
import pandas as pd 
movies=pd.read_csv(lpath+'\\movies.csv')
movies.columns

Index(['Movie ID', 'Title', 'Overview', 'Release Date', 'Runtime',
       'Popularity', 'Budget', 'Adult', 'Cast', 'Director',
       'Production Companies', 'Production Countries', 'Movie Homepage'],
      dtype='object')

In [2]:
movies.drop(labels=['Popularity', 'Budget'],axis=1,inplace=True)

In [3]:
movies.to_csv(lpath+'\\movies_data.csv',index=False)